In [ ]:
pip install yfinance

In [ ]:
pip install yahoo_fin

In [ ]:
pip install requests_html

     |████████████████████████████████| 133kB 2.8MB/s 
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.9 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pip install requests

In [ ]:
pip install bs4

In [ ]:
!pip install -U -q PyDrive

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from tabulate import tabulate
import datetime as dt#get current date
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from yahoo_fin.stock_info import get_quote_table
import requests
from bs4 import BeautifulSoup
import matplotlib.ticker as mtick
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
def Usr_input():
  previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
  Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
  years_ago = dt.date.today()-dt.timedelta(days=(5 * 365) + 30)
  Default_start = years_ago.strftime("%Y-%m-%d")
  Ticker = input('Please enter ticker(Default = JNJ)-->') or 'JNJ'
  Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
  End = input('Enter end date(Default = Yesterday, ' + Default_end + ')-->') or Default_end
  T_interval = input('Enter date interval(1d,1wk,1mo Default = 1mo)-->') or '1mo'
  return Ticker, Start, End, T_interval

In [ ]:
def calculate_ret(stock_a,stock_M,start,end_t,T_interval):
  Stock_price_a = yf.download(stock_a,start=start,end=end_t,interval = T_interval)['Adj Close'].dropna()
  Ret_a = Stock_price_a.pct_change()[1:]
  Stock_price_M = yf.download(stock_M,start=start,end=end_t,interval = T_interval)['Adj Close'].dropna()
  Ret_M = Stock_price_M.pct_change()[1:]
  Ret = pd.DataFrame([Ret_a,Ret_M])
  Ret = Ret.T
  Ret.columns = [stock_a,'SNP']
  return Ret

In [ ]:
def html_beautifulsoup(url):
    response = requests.get(url)
    html = response.content

    #beautifulsoup html will not look like soure
    #review beautifulsoup html to use find and findall
    soup = BeautifulSoup(html,"lxml")
    return soup

#soup_html: soup html is not same as source html
def yahoo_description(soup_html):
    element = soup_html.find('h1',class_='D(ib) Fz(16px) Lh(18px)')
    return element.text

In [ ]:
(Ticker,macro_start,macro_end,macro_T_interval) = Usr_input()
soup_html = html_beautifulsoup("https://finance.yahoo.com/quote/"  +Ticker)
Ticker_shortN = yahoo_description(soup_html)

Please enter ticker(Default = JNJ)-->FB
Enter start date(Default = 5 Years ago, 2015-05-28)-->2016-09-23
Enter end date(Default = Yesterday, 2020-06-24)-->2019-08-08
Enter date interval(1d,1wk,1mo Default = 1mo)-->1d


In [ ]:
ret = calculate_ret(Ticker,'^GSPC',macro_start,macro_end,macro_T_interval)
ret_M = ret['SNP']
ret_a = ret[Ticker]

ret_Mc = sm.add_constant(ret_M)
results = sm.OLS(ret_a,ret_Mc).fit()
print('                                 Market Model')
print(results.summary())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                                 Market Model
                            OLS Regression Results                            
Dep. Variable:                     FB   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     310.2
Date:                Thu, 25 Jun 2020   Prob (F-statistic):           5.33e-58
Time:                        00:57:07   Log-Likelihood:                 1987.9
No. Observations:                 721   AIC:                            -3972.
Df Residuals:                     719   BIC:                            -3963.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    s

In [ ]:
DW = np.sum( np.diff( results.resid.values )**2.0 ) / results.ssr
DW = round(DW, 3)
print(DW)

1.998


In [52]:
##Using significant level as 5 %, we find that dL is 1.20 and dU is 1.41.
dL = 1.2
dU = 1.41

if DW >0 and DW < dL:
  print("Durbin-Watson statistic(DW) is " + str(DW) + " , we have dL as 1.2, dU as 1.41. \nBecause DW value is greater than zero and less than dL, our test result is positive correlation")
elif DW > dL and DW < dU:
  print("Durbin-Watson statistic(DW) is " + str(DW) + " , we have dL as 1.2, dU as 1.41. \nBecause DW value is greater than dL and less than dU, our test result is inconclusive")
elif DW > dU and DW < 4 - dU:
  print("Durbin-Watson statistic(DW) is " + str(DW) + " , we have dL as 1.2, dU as 1.41. \nBecause DW value is greater than dU and less than 4 - dU, our test result is no correlation")
elif DW > 4 - dU and DW < 4 - dL:
  print("Durbin-Watson statistic(DW) is " + str(DW) + " , we have dL as 1.2, dU as 1.41. \nBecause DW value is greater than 4 - dU and less than 4 - dL, our test result is inconclusive")
else:
  print("Durbin-Watson statistic(DW) is " + str(DW) + " , we have dL as 1.2, dU as 1.41. \nBecause DW value is greater than 4 - dL and less than 4, our test result is negative correlation")

Durbin-Watson statistic(DW) is 1.998 , we have dL as 1.2, dU as 1.41. 
Because DW value is greater than dU and less than 4 - dU, our test result is no correlation
